## imports

In [1]:
from urllib.request import urlopen
import re
import json
import pandas as pd

### Number of Revisions

## choosing an API
Aparently, Wikipedia offers at least several outlets to the data it holds.

https://en.wikipedia.org/w/api.php
- stable
- seems to be a general-purpose wikipedia-maintenance tool
- many actions for page edition, inc. deletion, as well as for user management, like blocking

https://pl.wikipedia.org/api/rest_v1/

## getting revision ids

In [2]:
# from https://stackoverflow.com/a/34535903

def get_revisions(pageTitle):
    url = "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&rvlimit=500&titles=" + pageTitle
    url = "https://en.wikipedia.org/w/api.php?action=query&format=json&prop=revisions&rvlimit=500&rvprop=ids|timestamp|user|comment|content&titles=" + pageTitle
    # CHECK IT OUT 👀 https://api.wikimedia.org/core/v1/wikipedia/en/page/Earth/history?filter=bot&older_than=981126172
    # ...and this? https://en.wikipedia.org/api/rest_v1/ <-- official one?
    revisions = []                                        # list of all accumulated revisions
    next = ''                                             # information for the next request

    pages_revisions = {}

    while True:
      response = json.load(urlopen(url + next))      # web request
      revisions += response["query"]["pages"]  # adds all revisions from the current request to the list

      pages = [page for page in response["query"]["pages"].values()]
      print(json.dumps(pages))

      for page in pages:
        if not pages_revisions.get(page["title"]):
          pages_revisions[page["title"]] = {"revisions": []}

        pages_revisions[page["title"]]["revisions"].extend(page["revisions"])


      cont = response.get("continue", {}).get("rvcontinue", False)
      if not cont:
        break
      next = "&rvcontinue=" + cont

    return pages_revisions

In [3]:
revisions = get_revisions("Poland")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



KeyboardInterrupt: 

In [ ]:
print(json.dumps(revisions, indent=2))

In [ ]:
len(revisions["Poland"]["revisions"])

15251

In [ ]:
import json
print(json.dumps(revisions, indent=2))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import requests
import json

S = requests.Session()

DEFAUILT_TITLE = 'API|Main Page'

def get_revisions(title=DEFAUILT_TITLE):
  URL = "https://www.mediawiki.org/w/api.php"
  PARAMS = {
    "action": "query",
    "prop": "revisions",
    "titles": title,
    "rvprop": "timestamp|user|comment|content",
    "rvslots": "main",
    "formatversion": "2",
    "format": "json"
}
  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()
  PAGES = DATA["query"]["pages"]

  print(json.dumps(DATA, indent=2))

  for page in PAGES:
      # print(page["revisions"])
      print(json.dumps(page, indent=2))


revisions = get_revisions("German|Poland")

{
  "batchcomplete": true,
  "query": {
    "pages": [
      {
        "ns": 0,
        "title": "Poland",
        "missing": true
      },
      {
        "ns": 0,
        "title": "German",
        "missing": true
      }
    ]
  }
}
{
  "ns": 0,
  "title": "Poland",
  "missing": true
}
{
  "ns": 0,
  "title": "German",
  "missing": true
}


# Model

In [ ]:
!pip install llama-cpp-python

  Using cached llama_cpp_python-0.2.82.tar.gz (50.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.82-cp310-cp310-linux_x86_64.whl size=2833096 sha256=3cc5325e4b1daffa0eb8836c117cee298f21e28a9a1083544d322be39e0befc7
  Stored in directory: /root/.cache/pip/wheels/5f/da/5a/272c969ba31c678e6bee2543f3be4dfa67d0feb36ed6f94c01
Successfully built llama-cpp-python


In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama